In [1]:
from data.DeepCDRDataModule import DeepCDRDataModule
from models.DeepCDRLitModule import DeepCDRLitModule
from util.config import NeededFiles
from util.utils import get_preset_config
%load_ext tensorboard

## Configuration For Training

In [2]:
data = DeepCDRDataModule('data/raw', 100, batch_size=64)
model = DeepCDRLitModule(get_preset_config(), lr=0.001)

/home/dhruv.loke/DeepCDR-Reimplementation/venv/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


## Training The Model

In [4]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.lr_monitor import LearningRateMonitor

pl.seed_everything(0)
early_stopping = EarlyStopping(monitor="Validation loss", mode='min', stopping_threshold=0.005, patience=8)
logger=TensorBoardLogger(save_dir="log", name="DeepCDR_Logs", version="GAT Test With BCE Loss")
trainer = pl.Trainer(logger=logger, callbacks=[early_stopping], log_every_n_steps=1, gpus=1)
debug_trainer = pl.Trainer(max_epochs=2, gpus=1)

Global seed set to 0
/home/dhruv.loke/DeepCDR-Reimplementation/venv/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
trainer.fit(model, datamodule=data)

/home/dhruv.loke/DeepCDR-Reimplementation/venv/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory log/DeepCDR_Logs/GAT Test With BCE Loss/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name              | Type              | Params | In sizes              | Out sizes 
---------------------------------------------------------------------------------------------
0 | drug_gnn          | Sequential_5d1bca | 151 K  | [[200, 75], [2, 202]] | [200, 256]
1 | mutation_cnn1d    | Sequential        | 342 K  | [2, 1, 34673]         | [2, 100]  
2 | g_expr_fcn        | Sequential        | 204 K  | [2, 697]              | [2, 100]  
3 | meth_fcn          | Sequential        | 232 K  | [2, 808]              | [2, 100]  
4 | regression_conv1d | Sequential        | 239 K  | [2, 1, 25900]         | [2, 100]  
5 | regression_fc

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

../aten/src/ATen/native/cuda/Loss.cu:115: operator(): block: [0,0,0], thread: [0,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:115: operator(): block: [0,0,0], thread: [1,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:115: operator(): block: [0,0,0], thread: [2,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:115: operator(): block: [0,0,0], thread: [3,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:115: operator(): block: [0,0,0], thread: [4,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:115: operator(): block: [0,0,0], thread: [5,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:115: operator(): block: [0,0,0], thread: [6,0,0] Assertion `input_val >= zero && input_val <= one` 

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

: 

## Evaluation

In [10]:
# Loading TensorBoard
%reload_ext tensorboard
%tensorboard --logdir log/

In [11]:
gat_model = DeepCDRLitModule.load_from_checkpoint('log/DeepCDR_Logs/GAT Test/checkpoints/epoch=15-step=1520.ckpt')
# gcn_model = DeepCDRLitModule.load_from_checkpoint('log/DeepCDR_Logs/GCN Test/checkpoints/epoch=8-step=54.ckpt')
print("Testing GAT Model")
trainer.test(gat_model, data)
# print("Testing GCN Model")
# trainer.test(gcn_model, data)

/home/dhruv.loke/DeepCDR-Reimplementation/venv/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Testing GAT Model


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        Test loss           2.4507229328155518
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'Test loss': 2.4507229328155518}]